<h3>Задание 6. Задача об оптимальном использовании ресурсов</h3><br>Вариант 6

<b>Мат. модель задачи.</b><br>
Максимизация функции<br>
*F(x1,x2,x3) = (6b + 12)\*x1 + (5b + 22)\*x2 + c\*x3* → max<br><br>
при заданных ограничениях ресурсов:<br>
*x1 + 3\*x2 + a\*x3 <= 3000*<br>
*6\*x1 + 5\*x2 + 2\*x3 <= 3320*<br>
*x1, x2, x3 >= 0*

In [1]:
import numpy as np
import pandas as pd
from scipy.optimize import Bounds, linprog
import sympy as sym
import matplotlib.pyplot as plt
from gurobipy import *

In [2]:
a = 3
b = 1
c = 22

Оптимальный план производства

Linprog:

In [3]:
#границы
bounds = np.array([(0, np.inf), (0, np.inf), (0, np.inf)])

#ограничения

#A_ub - матрица коэф. при x1,x2,x3 в ограничениях, A_ub @ x <= b_ub
#c0 - коэффициенты перед переменными в целевой функции
A_ub = np.array([[1, 3, a], [6, 5, 2]])
b_ub = np.array([3000, 3320])
c0 = np.array([6*b + 12, 5*b + 22, c])

res = linprog(-c0, A_ub, b_ub, bounds=bounds, method='simplex', options={'disp': True})

x1 = res.x[0]
x2 = res.x[1]
x3 = res.x[2]
F_max = -res.fun
print(res.x)

Optimization terminated successfully.
         Current function value: -24640.000000
         Iterations: 3
[247.5   0.  917.5]


In [4]:
print(f'Оптимальные значения: x1 = {x1.round(3)}, x2 = {x2.round(3)}, x3 = {x3.round(3)}')
print('Max значение функции:',F_max.round(3))

Оптимальные значения: x1 = 247.5, x2 = 0.0, x3 = 917.5
Max значение функции: 24640.0


In [5]:
b_opt = np.array([x1 + 3*x2 + a*x3, 6*x1 + 5*x2 + 2*x3]) # Реальный расход ресурсов
b_opt

array([3000., 3320.])

In [6]:
#исчерпанность ресурсов
abs(b_ub-b_opt)

array([0., 0.])

Gurobi:

In [7]:
var_names = ['x1', 'x2', 'x3']                         
profit = np.array([6*b + 12, 5*b + 22, c])   # задаем цены
res_used = np.array([[1, 3, a], [6, 5, 2]])  # задаем использование ресурсов
res_avail = np.array([3000, 3320])           # задаем запасы

n = len(profit)                                    
m = len(res_avail)                                 
model = Model()
x = model.addVars(n, name=var_names) # заводим переменные в модель
obj = model.setObjective(quicksum(profit[j] * x[j] for j in range(n)), GRB.MAXIMIZE)
con = []
for i in range(m):
    con.append(model.addConstr(quicksum(res_used[i, j] * x[j] for j in range(n)) <= res_avail[i], name=var_names[i]))
model.optimize()

Restricted license - for non-production use only - expires 2023-10-25
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 2 rows, 3 columns and 6 nonzeros
Model fingerprint: 0xf7e02ca1
Coefficient statistics:
  Matrix range     [1e+00, 6e+00]
  Objective range  [2e+01, 3e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+03, 3e+03]
Presolve time: 0.01s
Presolved: 2 rows, 3 columns, 6 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    6.7000000e+31   3.375000e+30   6.700000e+01      0s
       2    2.4640000e+04   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.01 seconds (0.00 work units)
Optimal objective  2.464000000e+04


In [8]:
# параметры модели
print("X =", model.X, "\nFmax =", model.ObjVal, "\nОптимальный план двойственной задачи Pi → (y1,y2) =", model.Pi) 
print("x1 =", x[0].X)
print("Pi1 =", con[0].Pi)

X = [247.5, 0.0, 917.5] 
Fmax = 24640.0 
Оптимальный план двойственной задачи Pi → (y1,y2) = [6.0, 2.0000000000000004]
x1 = 247.5
Pi1 = 6.0


In [9]:
# Оценка устойчивости коэффициентов
model.printAttr(['X', 'Obj', 'SAObjLow', 'SAObjUp'])


    Variable            X          Obj     SAObjLow      SAObjUp 
----------------------------------------------------------------
          x1        247.5           18      16.2222           66 
          x2            0           27         -inf           28 
          x3        917.5           22      20.7692           54 


In [10]:
#Решение двойственной к M задачи
print('Анализ чувствительности\nFmax =', model.ObjVal)
model.printAttr(['X', 'RC', 'LB', 'SALBLow', 'SALBUp', 'UB', 'SAUBLow', 'SAUBUp'])
model.printAttr(['Sense', 'Slack', 'Pi', 'RHS', 'SARHSLow', 'SARHSUp'])

Анализ чувствительности
Fmax = 24640.0

    Variable            X           RC           LB      SALBLow       SALBUp           UB      SAUBLow       SAUBUp 
--------------------------------------------------------------------------------------------------------------------
          x1        247.5            0            0         -inf        247.5          inf        247.5          inf 
          x2            0           -1            0         -inf          440          inf            0          inf 
          x3        917.5            0            0         -inf        917.5          inf        917.5          inf 

  Constraint        Sense        Slack           Pi          RHS     SARHSLow      SARHSUp 
------------------------------------------------------------------------------------------
          x1            <            0            6         3000      553.333         4980 
          x2            <            0            2         3320         2000        18000 


Ответ. X∗ = (247.5; 0; 917.5); Fmax = F(X∗) = 24640